In [1]:
import tensorflow as tf
import requests
import base64
import json

2026-01-03 11:46:05.261544: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-01-03 11:46:05.261757: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/home/bryanh/miniconda3/envs/tfx_env/lib/python3.8/site-packages/google/api_core/_python_version_support.py:237: FutureWarning: You are using a non-supported Python version (3.8.20). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


## Test TF Serving

In [5]:
def prepare_json(features):
    # Extract individual feature values from the input list
    pregnancies, glucose, blood_pressure, skin_thickness, insulin, bmi, pedigree, age = features
    
    # Create a feature specification with individual features
    feature_spec = {
        "Pregnancies": tf.train.Feature(int64_list=tf.train.Int64List(value=[pregnancies])),
        "Glucose": tf.train.Feature(int64_list=tf.train.Int64List(value=[glucose])),
        "BloodPressure": tf.train.Feature(int64_list=tf.train.Int64List(value=[blood_pressure])),
        "SkinThickness": tf.train.Feature(int64_list=tf.train.Int64List(value=[skin_thickness])),
        "Insulin": tf.train.Feature(int64_list=tf.train.Int64List(value=[insulin])),
        "BMI": tf.train.Feature(float_list=tf.train.FloatList(value=[bmi])),
        "DiabetesPedigreeFunction": tf.train.Feature(float_list=tf.train.FloatList(value=[pedigree])),
        "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[age]))
    }
    
    # Create an Example protocol buffer
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
    
    # Encode the Example protocol buffer in base64
    result = [{'examples': {'b64': base64.b64encode(example).decode()}}]
    
    # Return the JSON formatted payload for the model
    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result
        }
    )

# Prepare the input numerical features to output diabetes presence
input_features = [6, 148, 72, 35, 0, 33.6, 0.527, 50]

# Make a POST request to the model server with the prepared JSON
response = requests.post(
    "http://localhost:8501/v1/models/serving_model:predict", 
    data=prepare_json(input_features),
    headers={"Content-Type": "application/json"}
).json()

print(response)

{'predictions': [[0.99999994]]}


## Test FastAPI

In [7]:
# Prepare the input numerical features to output no diabetes presence
input_features = [1, 85, 66, 29, 0, 26.6, 0.351, 31]

# Make a POST request to the model server with the prepared JSON
response = requests.post(
    "http://localhost:8080/v1/models/serving_model:predict", 
    data=prepare_json(input_features),
    headers={"Content-Type": "application/json"}
).json()

print(response)

{'outputs': [[1.0028332331213363e-10]]}


## Test Live Deployment (FastAPI)

In [7]:
# Prepare the input numerical features to output diabetes presence
input_features = [6, 148, 72, 35, 0, 33.6, 0.527, 50]

# Make a POST request to the model server with the prepared JSON
response = requests.post(
    "https://diabetes-detection-mlops.onrender.com/v1/models/serving_model:predict", 
    data=prepare_json(input_features),
    headers={"Content-Type": "application/json"}
).json()

print(response)

{'outputs': [[0.9999546408653259]]}
